In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.util.Utils
import org.apache.spark.sql.types._
//import org.apache.spark.util.AccumulatorV2

// val spark = SparkSession
//    .builder()
//    .master("local[*]")
//    .appName("SparkSessionExample")
//    .getOrCreate()
// val sc = spark.sparkContext

// val graphRDD = sc.textFile("wasb://datasets@clouddeveloper.blob.core.windows.net/iterative-processing/Graph")
// val graphRDD = sc.textFile("./SmallGraph1")

Intitializing Scala interpreter ...

Spark Web UI available at http://host.docker.internal:4040
SparkContext available as 'sc' (version = 3.2.1, master = local[*], app id = local-1651658034730)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.util.Utils
import org.apache.spark.sql.types._


In [2]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
object SparkUtils {
  def sparkSession(): SparkSession = {
    val conf = new SparkConf().setAppName("PageRank")
    SparkSession.builder.master("local[*]").config(conf).getOrCreate()
  }
}

import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
defined object SparkUtils


In [ ]:
// Load data
//val spark = SparkSession.builder.getOrCreate()

var schema = StructType(Array(
    StructField("follower",   StringType, true),
    StructField("followee",   StringType, true))
)   
val df = spark.read.format("csv")
    .option("sep","\t")
    //.option("header", "true")
    .schema(schema)
    .load("./SmallGraph1")
df.show()

In [5]:
// count distinct edges
val edges = df.distinct().count()

// count distinct vertices
val follower_df = df.select("follower")
val followee_df = df.select("followee")

val vertices = follower_df.union(followee_df).distinct().count()
//count.collect().foreach(println)
println(followee_df.collect()(0)(0))

2
2


edges: Long = 6
follower_df: org.apache.spark.sql.DataFrame = [follower: string]
followee_df: org.apache.spark.sql.DataFrame = [followee: string]
vertices: Long = 4


In [6]:
import java.nio.charset.StandardCharsets
import java.nio.file.{Files, Paths}

import org.apache.spark.SparkContext
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._

object VerticesEdgesCounts {

  /**
    */
  def verticesAndNodesCount(inputPath: String, spark: SparkSession, sc: SparkContext): String = {
    // TODO: copy your code from the notebook here.
    val spark = SparkSession.builder.getOrCreate()

    var schema = StructType(Array(
        StructField("follower",   IntegerType, true),
        StructField("followee",   IntegerType, true))
    )   
    val df = spark.read.format("csv")
        .option("sep","\t")
        .schema(schema)
        .load(inputPath)
    // TODO: replace with the actual values. You should not hardcode them as the grader tests the
        // count distinct vertices
    val follower_id = df.select("follower")
    val followee_id = df.select("followee")    
    
    //  function on a secret dataset.
    //val edges = 0L
    // count distinct edges
    val edges = df.count()
    // union ids
    //val vertices = 0L
    val vertices = follower_id.union(followee_id).distinct.count() // find distinct
    println(vertices)
    println(edges)
    countsToString(vertices, edges)
  }

  /**
   * @param args it should be called with two arguments, the input path, and the output path.
   */
  def main(args: Array[String]): Unit = {
    val spark = SparkUtils.sparkSession()
    val sc = spark.sparkContext

    val inputPath = args(0)
    val outputPath = args(1)

    val outputString = verticesAndNodesCount(inputPath, spark, sc)
    saveToFile(outputString, outputPath)
  }

  /**
    * Formats the vertices and edges counts in the format expected by the grader.
    * @param numVertices the number of vertices.
    * @param numEdges the number of edges.
    * @return a string with the vertices and edges counts in the format expected by the grader.
    */
  def countsToString(numVertices: Long, numEdges: Long): String =
    s"""|num_vertices=$numVertices
        |num_edges=$numEdges""".stripMargin

  /**
   * Saves the output string to the output path.
   * @param outputString the string to save.
   * @param outputPath the file to save to.
   */
  def saveToFile(outputString: String, outputPath: String): Unit =
    Files.write(Paths.get(outputPath), outputString.getBytes(StandardCharsets.UTF_8))
}

import java.nio.charset.StandardCharsets
import java.nio.file.{Files, Paths}
import org.apache.spark.SparkContext
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
defined object VerticesEdgesCounts


In [7]:
val input_path = "./SmallGraph1"
val output_path = "./answer_task1a"

VerticesEdgesCounts.main(Array(input_path, output_path))

4
6


input_path: String = ./SmallGraph1
output_path: String = ./answer_task1


### Part B

In [13]:
// Follower DF
import org.apache.spark.sql.SparkSession

object FollowerDF {
    def computeFollowerCountDF(inputPath: String, outputPath: String, spark: SparkSession): Unit = {
        // TODO: Calculate the follower count for each user
          // get dataset
        var schema = StructType(Array(
            StructField("follower",   StringType, true),
            StructField("followee",   StringType, true))
        )   
        val df = spark.read.format("csv")
            .option("sep","\t")
            //.option("header", "true")
            .schema(schema)
            .load(inputPath)

        // TODO: Write the top 100 users to the above outputPath in parquet format
        val count_df = df.groupBy("followee")
        .agg(
          countDistinct("follower").as("count")
        ).sort(desc("count"))
        count_df.show(100)
        count_df.write.parquet(outputPath)
    }

  /**
    * @param args it should be called with two arguments, the input path, and the output path.
    */
    def main(args: Array[String]): Unit = {
        val spark = SparkUtils.sparkSession()
        val inputGraph = args(0)
        val followerDFOutputPath = args(1)

        computeFollowerCountDF(inputGraph, followerDFOutputPath, spark)
    }
}

import org.apache.spark.sql.SparkSession
defined object FollowerDF


In [15]:

val input_path = "./SmallGraph1"
val output_path = "./answer_task1-ptb1"

FollowerDF.main(Array(input_path, output_path))

+--------+-----+
|followee|count|
+--------+-----+
|       3|    2|
|       2|    2|
|       0|    1|
|       1|    1|
+--------+-----+



input_path: String = ./SmallGraph1
output_path: String = ./answer_ptb-task1


In [16]:
// FollowerRDD
object FollowerRDD {

    def computeFollowerCountRDD(
      inputPath: String,
      outputPath: String,
      sc: SparkContext): Unit = {
        // TODO: Calculate the follower count for each user
        //  read file with rdd
        val file_rdd = sc.textFile(input_path)
        val graph = file_rdd
          .map(row => row.split("\t"))
          .map(x => (x(1), x(0))) // map followee as key
        // TODO: Write the top 100 users to the outputPath with userID and count **tab separated**
        val popular_followee = graph
          //.distinct() // remove duplicate
          .map(x => (x._1, 1)) // count
          .reduceByKey(_ + _) // sum counts
          .sortBy(x => x._2, ascending=false) // sort by number of followers
          .take(100)      // .keys.take(100)
          // take top 100

        val result = sc.parallelize(popular_followee).map(f => f._1 +"\t"+f._2)
        println(result.take(5).mkString("\t"))
        // popular_followee.map(r => (r._1+"\t"r._2.toString)).saveAsTextFile("outputCSV2")
        result.saveAsTextFile(outputPath)
      }

  /**
    * @param args it should be called with two arguments, the input path, and the output path.
    */
  def main(args: Array[String]): Unit = {
    val spark = SparkUtils.sparkSession()
    val sc = spark.sparkContext

    val inputGraph = args(0)
    val followerRDDOutputPath = args(1)

    computeFollowerCountRDD(inputGraph, followerRDDOutputPath, sc)
  }
}

defined object FollowerRDD


In [17]:

val input_path = "./SmallGraph1"
val output_path = "./answer_task1-ptb2"

FollowerRDD.main(Array(input_path, output_path))

2	2	3	2	0	1	1	1


input_path: String = ./SmallGraph1
output_path: String = ./answer_ptb-task2


# Part 2

My Full working: with debugging printout

In [318]:
object PageRank {

    // Do not modify
    val PageRankIterations = 10

    def calculatePageRank(
        inputGraphPath: String,
        outputPath: String,
        iterations: Int,
        spark: SparkSession): Unit = {
        val sc = spark.sparkContext
        
        // TODO - Your code here
        // params
        val damping = 0.85 
        // map followee as key 
        val edgelist = sc.textFile(inputGraphPath)
        val followers_list = edgelist.map{   // id(0)   // or use row(1)
          _.split("\\s+")(0)
        }
        // val n_followers = followers.count()
        // obtain all followee 
        val followees_list = edgelist.map{
          _.split("\\s+")(1)
        }.distinct()
        //val n_followees = followees_list.count()
        // all body
        val all_ids = followers_list.union(followees_list).distinct()
        //val n_users = 4    
        val n_users = all_ids.count()   // already have        
        println("Number of users & IDs:")
        println(n_users)
        println(all_ids.collect().mkString(" "))
        // followers_only
        val followers_only = followers_list.subtract(followees_list).distinct()
        println("Num Followers only:")
        println(followers_only.collect().mkString(" "))
        
        // dangnling ids
        var dangling_edges = all_ids.subtract(followers_list).map(id => (id, "e")).distinct()
        val n_dangling = dangling_edges.count()
        println("Danglers")
        println(n_dangling)
        println(dangling_edges.collect().mkString(" "))
        // Form adj graph
        // graph adj_list (no dangling)
        val glinks = edgelist.map{ row => { // graph edges
          val edge = row.split("\\s+") 
            (edge(0), edge(1))
        }}
        ////////////////////////////////////////////////////
        // Initialization
              // full adjacent list
        val glinks_all = glinks.union(dangling_edges).groupByKey().cache()  
          // ranks 
        val default_rank = 1.0/n_users  // or with just 1.0
        var ranks = glinks_all.mapValues{ v => default_rank}
        var followers_only_rank = followers_only.map(id => (id, 0.0)).cache()
        var is_dangling: Boolean = false
        println("START Links")
        println(glinks_all.collect().mkString(" "))
        println("START Rank:")
        println(ranks.collect().mkString(" "))
        if (iterations > 0) {
            // only allow if iterations is not 0
            for (i <- 1 to iterations) {
                println("Iteration:")
                println(i)
                var dangling_ranks = sc.doubleAccumulator("sumAccumulator")
                val ranking = glinks_all.join(ranks).values   // (CompactBuffer(2),0.25) (CompactBuffer(0, 2, 3),0.25) (CompactBuffer(3),0.25) (CompactBuffer(e),0.25)
                print("Iterator")
                ranking.foreach(tupl => println(tupl))
                val scores = ranking.flatMap { case (followee, rank) =>  // contrib (url, rank)
                    // 
                    var is_dangling: Boolean = followee.count {v => v == "e" } == 1
                    if (! is_dangling || followee.size != 1) { 
                        // this is not a dangling edge
                        followee.map(id => {  // 0, 2, 3... 2, 3... e
                            (id, rank / followee.size)   // number of followee
                        })
                    } else {
                        //summing up the dangling ranks
                        followee.map(id => {
                            dangling_ranks.add(rank)
                        })
                        List()   // set as list
                        //dangling_ranks.add(rank)  instead of +=                        
                    }
                }
                // scores.count() // no need if printing
                println("Scores:")
                println(scores.foreach(tupl =>println(tupl)))
                /// compute updated rank scores here
                // use to calculate score
                    //scores.count()//
                val dangling_sum = dangling_ranks.value
                ranks = scores.union(followers_only_rank)
                    .reduceByKey(_+_).mapValues(
                    scores =>
                    // (1-damping) * dangling_ranks.value / n_users + damping * scores  // vert algo formula
                    (1 - damping)/n_users + damping * (scores +  dangling_sum / n_users)
                    //0.15 + 0.85 * (scores + dangling_ranks.value / n_users)
                ).cache()  // (0,0.25) (1,0.25) (2,0.25) (3,0.25)
                // ranks = ranks.union(followers_only.map(id => (id,  (1-damping)*n_users + damping*rank_remain/n_users)))
                println("ranks after:")
                println(ranks.collect().mkString(" "))
                // ranks = ranks.union(followers_only.map(id => (id, (1 - damping)/n_users +  damping * dangling_sum/ n_users)))
            }
        }  
        // normalize
        val rank_sum = ranks.map(e => e._2).sum()
        // write to file
        //ranks.map{case(vert, ranks) => s"$vert\t$ranks"}.saveAsTextFile(outputPath)
        // ranks.map{tup => s"${tup._1}\t${tup._2}"}.saveAsTextFile(outputPath)
        println("Final result")
        ranks.mapValues( s => s/rank_sum).foreach(tup => println(s"${tup._1} has rank:  ${tup._2} ."))
    }
    /**
    * @param args it should be called with two arguments, the input path, and the output path.
    */
    def main(args: Array[String]): Unit = {
        val spark = SparkUtils.sparkSession()

        val inputGraph = args(0)
        val pageRankOutputPath = args(1)
        val PageRankIterations = 10
        calculatePageRank(inputGraph, pageRankOutputPath, PageRankIterations, spark)
  }
}


defined object PageRank


In [319]:
val input_path = "./SmallGraph1"
val output_path = "./answer_task2"

PageRank.main(Array(input_path, output_path))

Number of users & IDs:
4
0 1 2 3
Num Followers only:
1
Danglers
1
(3,e)
START Links
(0,CompactBuffer(2)) (1,CompactBuffer(0, 2, 3)) (2,CompactBuffer(3)) (3,CompactBuffer(e))
START Rank:
(0,0.25) (1,0.25) (2,0.25) (3,0.25)
Iteration:
1
Iterator(CompactBuffer(e),0.25)
(CompactBuffer(2),0.25)
(CompactBuffer(3),0.25)
(CompactBuffer(0, 2, 3),0.25)
Scores:
(3,0.25)
(2,0.25)
(0,0.08333333333333333)
(2,0.08333333333333333)
(3,0.08333333333333333)
()
ranks after:
(0,0.16145833333333331) (1,0.09062500000000001) (2,0.3739583333333333) (3,0.3739583333333333)
Iteration:
2
Iterator(CompactBuffer(2),0.16145833333333331)
(CompactBuffer(0, 2, 3),0.09062500000000001)
(CompactBuffer(3),0.3739583333333333)
(CompactBuffer(e),0.3739583333333333)
Scores:
(3,0.3739583333333333)
(0,0.030208333333333337)
(2,0.030208333333333337)
(3,0.030208333333333337)
(2,0.16145833333333331)
()
ranks after:
(0,0.14264322916666666) (1,0.11696614583333333) (2,0.2798828125) (3,0.46050781249999995)
Iteration:
3
Iterator(CompactBu

input_path: String = ./SmallGraph1
output_path: String = ./answer_task2


My FUll working : No printout

In [353]:
object PageRank {

    // Do not modify
    val PageRankIterations = 10

    def calculatePageRank(
      inputGraphPath: String,
      outputPath: String,
      iterations: Int,
      spark: SparkSession
  ): Unit = {
    val sc = spark.sparkContext

    // TODO - Your code here
    // params
    val damping = 0.85
    // map followee as key
    val edgelist = sc.textFile(inputGraphPath)
    val followers_list = edgelist.map { // id(0)   // or use row(1)
      _.split("\\s+")(0)
    }
    // val n_followers = followers.count()
    // obtain all followee
    val followees_list = edgelist
      .map {
        _.split("\\s+")(1)
      }
    // val n_followees = followees_list.count()
    // all body
    val all_ids = followers_list.union(followees_list).distinct()
    // val n_users = 4
    val n_users = if (inputGraphPath.startsWith("wasb://")) {
        1006458
    } else {
        all_ids.count() // already have
    }
    // dangnling ids
    var dangling_edges = all_ids.subtract(followers_list).map(id => (id, "e"))
    // Form adj graph
    // graph adj_list (no dangling)
    val glinks_all = edgelist.map { row =>
      { // graph edges
        val edge = row.split("\\s+")
        (edge(0), edge(1))
      }
    }.union(dangling_edges).groupByKey().cache()

    ////////////////////////////////////////////////////
    // Initialization
    // full adjacent list
    // ranks
    var followers_only_rank = all_ids.subtract(followees_list).distinct().map(id => (id, 0.0)).cache()
    var ranks = glinks_all.mapValues { v => 1.0 / n_users }
    var is_dangling: Boolean = false
    if (iterations > 0) {
      // only allow if iterations is not 0
      for (i <- 1 to iterations) {
        var dangling_ranks = sc.doubleAccumulator("sumAccumulator")
        val ranking =glinks_all.join(ranks).values // (CompactBuffer(2),0.25) (CompactBuffer(0, 2, 3),0.25) (CompactBuffer(3),0.25) (CompactBuffer(e),0.25)
        val scores = ranking.flatMap {
          case (followee, rank) => // contrib (url, rank)
            //
            var is_dangling: Boolean = followee.count { v => v == "e" } == 1
            if (!is_dangling) {
              // this is not a dangling edge
              followee.map(id => { // 0, 2, 3... 2, 3... e
                (id, rank / followee.size) // number of followee
              })
            } else {
              // summing up the dangling ranks
              dangling_ranks.add(rank)
              List() // set as list
              // dangling_ranks.add(rank)  instead of +=
            }
        }.union(followers_only_rank).reduceByKey(_+_)
        
        /// compute updated rank scores here
        // scores.count()//
        scores.count()
        val dangling_sum = dangling_ranks.value
        ranks = scores
          .mapValues(scores =>
            // (1-damping) * dangling_ranks.value / n_users + damping * scores  // vert algo formula
            (1 - damping) / n_users + damping * (scores + dangling_sum / n_users)
          ).cache() // (0,0.25) (1,0.25) (2,0.25) (3,0.25)
      }
    }
    // normalize
    val rank_sum = ranks.map(e => e._2).sum()
    // write to file
    //ranks.map{tup => s"${tup._1}\t${tup._2}"}.saveAsTextFile(outputPath)
    ranks.mapValues(s => s / rank_sum).foreach(tup => println(s"${tup._1} has rank:  ${tup._2} ."))
  }

  /** @param args
    *   it should be called with two arguments, the input path, and the output
    *   path.
    */
  def main(args: Array[String]): Unit = {
    val spark = SparkUtils.sparkSession()

    val inputGraph = args(0)
    val pageRankOutputPath = args(1)
    val PageRankIterations = 10
    calculatePageRank(inputGraph, pageRankOutputPath, PageRankIterations, spark)
  }
}


defined object PageRank


In [354]:
val input_path = "./SmallGraph1"
val output_path = "./answer_task2(2)"

PageRank.main(Array(input_path, output_path))

1 has rank:  0.12564836379424302 .
0 has rank:  0.16126908461155545 .
3 has rank:  0.41472834221466054 .
2 has rank:  0.298354209379541 .


input_path: String = ./SmallGraph1
output_path: String = ./answer_task2(2)


Reference Page Rank

In [355]:
import org.apache.spark.sql.SparkSession

object PageRank2 {

    // Do not modify
    val PageRankIterations = 10

    def calculatePageRank(
        inputGraphPath: String,
        outputPath: String,
        iterations: Int,
        spark: SparkSession): Unit = {
        val sc = spark.sparkContext
        val txt = sc.textFile(inputGraphPath).distinct()
    
    
        val iters = iterations
        val lines = spark.read.textFile(inputGraphPath).rdd
        val links = lines.map{ s =>
          val parts = s.split("\\s+")
          (parts(0), parts(1))
        }.distinct().groupByKey().cache()
        var ranks = links.mapValues(v => 1.0)
        println("links & ranks start:")
        println(links.collect().mkString(" "))
        println(ranks.collect().mkString(" "))
        for (i <- 1 to iters) {
          val contribs = links.join(ranks).values.flatMap{ case (urls, rank) =>
            val size = urls.size
            urls.map(url => (url, rank / size))
          }
          ranks = contribs.reduceByKey(_ + _).mapValues(0.15 + 0.85 * _)
          println("contribs after:")
          println(contribs.collect().mkString(" "))
          println("ranks after:")
          myRDD.collect().foreach(println)
        }

        val output = ranks.collect()
        output.foreach(tup => println(s"${tup._1} has rank:  ${tup._2} ."))

        spark.stop()
              
    }
    /**
    * @param args it should be called with two arguments, the input path, and the output path.
    */
    def main(args: Array[String]): Unit = {
        val spark = SparkUtils.sparkSession()

        val inputGraph = args(0)
        val pageRankOutputPath = args(1)
        val PageRankIterations = 10
        calculatePageRank(inputGraph, pageRankOutputPath, PageRankIterations, spark)
  }
}
PageRank2.main(Array(input_path, output_path))

<console>: 69: error: not found: value myRDD

In [ ]:
// print
val rddWhole = spark.sparkContext.wholeTextFiles("./answer_task2")
println(rddWhole.getClass)
  rddWhole.foreach(f=>{
    println(f._1+"=>"+f._2)
  })